In [2]:
'/Users/macbook/Downloads/exports_10-07-2025 3/10-07-2025_ranking_2025-05-01 00:00:00_2025-05-01 00:00:00.xlsx'
import pandas as pd

# File paths
ranking_file = '/Users/macbook/Downloads/exports_10-07-2025 3/10-07-2025_ranking_2025-05-01 00:00:00_2025-05-01 00:00:00.xlsx'
profile_file = '/Users/macbook/Downloads/exports_10-07-2025 3/10-07-2025_profile_2025-05-01 00:00:00_2025-05-01 00:00:00.xlsx'

# Read Excel files
df_ranking = pd.read_excel(ranking_file)
df_profile = pd.read_excel(profile_file)

# Combine on 'tool_id'
df_merged = pd.merge(df_ranking, df_profile, on='tool_id', how='inner')



In [3]:
# Count how many tool names end with 'ai' (case-insensitive)
num_tools_end_with_ai = df_merged['name_tool'].dropna().apply(lambda x: str(x).strip().lower().endswith('ai')).sum()
print(f"Number of tools whose name ends with 'ai': {num_tools_end_with_ai}")


Number of tools whose name ends with 'ai': 3645


In [1]:
df_merged.head()

NameError: name 'df_merged' is not defined

In [3]:
# Keep only the specified columns: tool_id, use_original, use aggreagted, name_tool
# Note: column names are case-sensitive and may need to be adjusted if they differ in the actual DataFrame
columns_to_keep = ['tool_id', 'use_original', 'use_aggregated', 'name_tool', 'tool_url_y', 'tool_description']
df_selected = df_merged[columns_to_keep]
df_selected.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
0,744,;prototype development;ui/ux design & optimiza...,prototype development;ui/ux design & optimizat...,Figma,figma.com,Figma is a collaborative design platform enabl...
1,12052,;data analysis;data visualization;data governance,;data analysis;data visualization;data governance,DataBricks,databricks.com,Databricks offers an integrated platform for d...
2,12057,;foundation models (llm),foundation models (llm),Grok,x.ai,Grok-2 offers advanced conversational AI and p...
3,6660,;foundation models (llm),foundation models (llm),ChatGPT,chatgpt.com,Multimodal chat service exposes large language...
4,778,;payroll management;compliance management;cont...,payroll management;compliance management;contr...,Deel,deel.com,Deel provides a comprehensive platform for glo...


In [4]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19238 entries, 0 to 19237
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           19238 non-null  int64 
 1   use_original      11537 non-null  object
 2   use_aggregated    11339 non-null  object
 3   name_tool         19238 non-null  object
 4   tool_url_y        19238 non-null  object
 5   tool_description  13568 non-null  object
dtypes: int64(1), object(5)
memory usage: 901.9+ KB


In [140]:
# Find a couple of rows where 'use_original' and 'use_aggregated' are different
diff_rows = df_selected[df_selected['use_original'] != df_selected['use_aggregated']]
# Display the first few such rows
diff_rows.sample(5)


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
13988,16588,NaN,NaN,IELTS Podcast,essaycheck.ieltspodcast.co,NaN
10570,100,NaN,NaN,Altera AI,altera.ai,Altera creates digital human beings with AI ar...
13499,22566,NaN,NaN,FILL GENIUS,fillgenius.com,FILL GENIUS automates the process of completin...
14703,20021,NaN,NaN,StoryDiffusion AI,storydiffusion.org,NaN
4745,21453,NaN,NaN,The Fabricant,thefabricant.ai,"Digital‑fashion house creating 3D garments, NF..."


In [6]:

from typing import List, Optional, Union
import pandas as pd

def filter_by_keywords(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    case_sensitive: bool = False
) -> pd.DataFrame:
    """
    Filter DataFrame rows where the specified column contains any of the given keywords.
    """
    def contains_keywords(text):
        if not isinstance(text, str):
            return False
        text_cmp = text if case_sensitive else text.lower()
        kw_list = keywords if case_sensitive else [kw.lower() for kw in keywords]
        return any(kw in text_cmp for kw in kw_list)
    return df[df[column].apply(contains_keywords)]

def remove_keywords_from_column(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    tool_ids: Optional[List[int]] = None
) -> pd.DataFrame:
    """
    Remove specified keywords from a column, optionally only for certain tool_ids.
    """
    def remove_keywords(text):
        if not isinstance(text, str):
            return text
        result = text
        for kw in keywords:
            # Remove keyword with or without leading/trailing semicolons and extra spaces
            result = result.replace(f";{kw}", "")
            result = result.replace(f"{kw};", "")
            result = result.replace(kw, "")
        # Clean up any accidental double semicolons or leading/trailing semicolons
        result = result.replace(";;", ";").strip(";").strip()
        return result

    mask = df['tool_id'].isin(tool_ids) if tool_ids is not None else pd.Series([True]*len(df), index=df.index)
    df.loc[mask, column] = df.loc[mask, column].apply(remove_keywords)
    return df

def add_missing_tool_ids(
    filtered_df: pd.DataFrame,
    df_selected: pd.DataFrame,
    tool_ids_to_add: List[int]
) -> pd.DataFrame:
    """
    Add rows from df_selected to filtered_df for tool_ids not already present.
    """
    existing_tool_ids = set(filtered_df['tool_id'])
    missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]
    missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]
    return pd.concat([filtered_df, missing_rows], ignore_index=True)


### Software development

In [44]:
# Find all rows where 'use_original' contains any of the specified keywords
keywords = [
    'app development',
    'software development',
    'website development',
]

def contains_keywords_use_original(text):
    text = str(text).lower()
    return any(keyword in text for keyword in keywords)

# Filter the DataFrame for relevant rows
filtered_df = df_selected[df_selected['use_original'].apply(contains_keywords_use_original)]




In [45]:
# Find all rows where 'tool_description' contains 'software development' (case-insensitive)
desc_mask = df_selected['tool_description'].astype(str).str.lower().str.contains('app development', na=False)
desc_software_dev_df = df_selected[desc_mask]

# Display the first few such rows
desc_software_dev_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 166 to 17122
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           37 non-null     int64 
 1   use_original      33 non-null     object
 2   use_aggregated    33 non-null     object
 3   name_tool         37 non-null     object
 4   tool_url_x        37 non-null     object
 5   tool_description  37 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.0+ KB


In [46]:
# List of tool_ids to add if not already in filtered_df
tool_ids_to_add = [
    13653, 13360, 6052, 15514, 5467, 13189, 11563, 4853, 8901, 5013, 9134, 1843,
    10731, 19342, 6126, 1385, 469, 19680, 11257, 1474, 8419
]

# Find which tool_ids are not already in filtered_df
existing_tool_ids = set(filtered_df['tool_id'])
missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]

# Get the rows from df_selected for these missing tool_ids
missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]

# Append these rows to filtered_df
filtered_df = pd.concat([filtered_df, missing_rows], ignore_index=True)



In [47]:
# Save filtered_df to Excel file
desc_software_dev_df.to_excel('filtered_software_development_tools.xlsx', index=False)


In [48]:
filtered_df.sample(5)
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           299 non-null    int64 
 1   use_original      297 non-null    object
 2   use_aggregated    297 non-null    object
 3   name_tool         299 non-null    object
 4   tool_url_x        299 non-null    object
 5   tool_description  298 non-null    object
dtypes: int64(1), object(5)
memory usage: 14.1+ KB


In [49]:

# Add 'software development' to use_aggregated for these tools, if not already present
def add_software_development(agg):
    agg = str(agg) if pd.notnull(agg) else ''
    # Remove leading/trailing semicolons and whitespace
    agg_clean = agg.strip(';').strip()
    # Check if 'software development' is already present (case-insensitive)
    if 'software development' in agg_clean.lower().split(';'):
        return agg_clean
    # Add 'software development' at the end, separated by semicolon if needed
    if agg_clean:
        return agg_clean + ';software development'
    else:
        return 'software development'

# Update the original df_selected DataFrame in place
for idx in filtered_df.index:
    current_agg = df_selected.at[idx, 'use_aggregated']
    df_selected.at[idx, 'use_aggregated'] = add_software_development(current_agg)

filtered_df = df_selected.loc[filtered_df.index]  # Refresh filtered_df with updated values

filtered_df.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
0,744,;prototype development;ui/ux design & optimiza...,prototype development;ui/ux design & optimizat...,Figma,https://figma.com,Figma is a collaborative design platform enabl...
1,12052,;data analysis;data visualization;data governance,data analysis;data visualization;data governan...,DataBricks,https://databricks.com,Databricks offers an integrated platform for d...
2,12057,;foundation models (llm),foundation models (llm);software development,Grok,https://x.ai,Grok-2 offers advanced conversational AI and p...
3,6660,;foundation models (llm),foundation models (llm);software development,ChatGPT,https://chatgpt.com,Multimodal chat service exposes large language...
4,778,;payroll management;compliance management;cont...,payroll management;compliance management;contr...,Deel,https://deel.com,Deel provides a comprehensive platform for glo...


In [50]:
# Save filtered_df to Excel with 'tool_url_x' renamed to 'tool_url'
filtered_df_to_save = filtered_df.rename(columns={'tool_url_x': 'tool_url'})
filtered_df_to_save.to_excel('software_development_sub_ranking.xlsx', index=False)


### Energy management, monitoring & analysis

In [71]:
# Find all rows where 'use_original' contains any of the specified keywords
keywords = [
"gaming"
]

def contains_keywords_use_original(text):
    text = str(text).lower()
    return any(keyword in text for keyword in keywords)

# Filter the DataFrame for relevant rows
filtered_df_energy = df_selected[df_selected['use_original'].apply(contains_keywords_use_original)]

# Save filtered_df to Excel file

In [72]:
filtered_df_energy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 1017 to 19234
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           34 non-null     int64 
 1   use_original      34 non-null     object
 2   use_aggregated    34 non-null     object
 3   name_tool         34 non-null     object
 4   tool_url_x        34 non-null     object
 5   tool_description  34 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.9+ KB


In [55]:
# List of tool_ids for which to delete the specified keywords
tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]
keywords_to_remove = [
    "energy monitoring & analysis", "energy management", "climate control"
]

def remove_keywords_from_aggregated(text):
    if not isinstance(text, str):
        return text
    result = text
    for kw in keywords_to_remove:
        # Remove keyword with or without leading/trailing semicolons and extra spaces
        result = result.replace(f";{kw}", "")
        result = result.replace(f"{kw};", "")
        result = result.replace(kw, "")
    # Clean up any accidental double semicolons or leading/trailing semicolons
    result = result.replace(";;", ";").strip(";").strip()
    return result

mask = filtered_df_energy['tool_id'].isin(tool_ids_to_edit)
filtered_df_energy.loc[mask, 'use_aggregated'] = (
    filtered_df_energy.loc[mask, 'use_aggregated'].apply(remove_keywords_from_aggregated)
)



In [56]:
filtered_df_energy.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
1704,12745,;energy optimization;carbon footprint analysis...,;energy optimization;carbon footprint analysis...,metron,https://metron.energy,The METRON Energy Management and Optimization ...
2232,13057,;energy management;heating control,;energy management;heating control,Mysa,https://getmysa.com,Mysa Smart Thermostats offer control over elec...
2894,3340,;transaction monitoring;app development;energy...,transaction monitoring;app development;pricing...,MeetFrank,https://meetfrank.com,"A comprehensive platform offering remote, hybr..."
3982,5511,;energy optimization;compliance management;ene...,;energy optimization;compliance management;ene...,Ecoplanet,https://ecoplanet.tech,Ecoplanet offers an advanced energy management...
5857,4653,;energy monitoring & analysis;energy management,;energy monitoring & analysis;energy management,etalytics,https://etalytics.com,etalytics provides AI-driven energy management...


In [57]:
filtered_df_energy.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 1704 to 19180
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           20 non-null     int64 
 1   use_original      20 non-null     object
 2   use_aggregated    20 non-null     object
 3   name_tool         20 non-null     object
 4   tool_url_x        20 non-null     object
 5   tool_description  20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB


In [54]:
#to excel
filtered_df_energy.to_excel('filtered_energy_tools.xlsx', index=False)

In [61]:
# Find all rows where 'tool_description' contains 'software development' (case-insensitive)
desc_mask = df_selected['tool_description'].astype(str).str.lower().str.contains('energy management', na=False)
energy_df = df_selected[desc_mask]

# Display the first few such rows
energy_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 1704 to 19180
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           13 non-null     int64 
 1   use_original      10 non-null     object
 2   use_aggregated    10 non-null     object
 3   name_tool         13 non-null     object
 4   tool_url_x        13 non-null     object
 5   tool_description  13 non-null     object
dtypes: int64(1), object(5)
memory usage: 728.0+ bytes


In [ ]:
# Save filtered_df to Excel file
desc_software_dev_df.to_excel('filtered_software_development_tools.xlsx', index=False)

In [ ]:
# List of tool_ids to add if not already in filtered_df
tool_ids_to_add = [
    13653, 13360, 6052, 15514, 5467, 13189, 11563, 4853, 8901, 5013, 9134, 1843,
    10731, 19342, 6126, 1385, 469, 19680, 11257, 1474, 8419
]

# Find which tool_ids are not already in filtered_df
existing_tool_ids = set(filtered_df['tool_id'])
missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]

# Get the rows from df_selected for these missing tool_ids
missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]

# Append these rows to filtered_df
filtered_df = pd.concat([filtered_df, missing_rows], ignore_index=True)



## Gaming


In [141]:

from typing import List, Optional, Union
import pandas as pd

def filter_by_keywords(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    case_sensitive: bool = False
) -> pd.DataFrame:
    """
    Filter DataFrame rows where the specified column contains any of the given keywords.
    """
    def contains_keywords(text):
        if not isinstance(text, str):
            return False
        text_cmp = text if case_sensitive else text.lower()
        kw_list = keywords if case_sensitive else [kw.lower() for kw in keywords]
        return any(kw in text_cmp for kw in kw_list)
    return df[df[column].apply(contains_keywords)]

def remove_keywords_from_column(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    tool_ids: Optional[List[int]] = None
) -> pd.DataFrame:
    """
    Remove specified keywords from a column, optionally only for certain tool_ids.
    """
    def remove_keywords(text):
        if not isinstance(text, str):
            return text
        result = text
        for kw in keywords:
            # Remove keyword with or without leading/trailing semicolons and extra spaces
            result = result.replace(f";{kw}", "")
            result = result.replace(f"{kw};", "")
            result = result.replace(kw, "")
        # Clean up any accidental double semicolons or leading/trailing semicolons
        result = result.replace(";;", ";").strip(";").strip()
        return result

    mask = df['tool_id'].isin(tool_ids) if tool_ids is not None else pd.Series([True]*len(df), index=df.index)
    df.loc[mask, column] = df.loc[mask, column].apply(remove_keywords)
    return df

def add_missing_tool_ids(
    filtered_df: pd.DataFrame,
    df_selected: pd.DataFrame,
    tool_ids_to_add: List[int]
) -> pd.DataFrame:
    """
    Add rows from df_selected to filtered_df for tool_ids not already present.
    """
    existing_tool_ids = set(filtered_df['tool_id'])
    missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]
    missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]
    return pd.concat([filtered_df, missing_rows], ignore_index=True)


In [151]:

# Define your keywords and tool_ids for any use case
keywords = [
    "gaming"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids
#keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df_gaming = filter_by_keywords(df_selected, 'use_original', keywords)

In [152]:
filtered_df_gaming.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 1017 to 19234
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           34 non-null     int64 
 1   use_original      34 non-null     object
 2   use_aggregated    34 non-null     object
 3   name_tool         34 non-null     object
 4   tool_url_y        34 non-null     object
 5   tool_description  34 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.9+ KB


In [153]:
# 2. Remove keywords from 'use_aggregated' for certain tool_ids
filtered_df_gaming = remove_keywords_from_column(filtered_df_gaming, 'use_aggregated', keywords_to_remove, tool_ids=tool_ids_to_edit)

In [146]:

# 3. Save to Excel (filename can be parameterized)
filtered_df_gaming.to_excel('filtered_tools.xlsx', index=False)


In [147]:

# 4. Filter by keywords in 'tool_description' (example: for "software development" or any other)
desc_keywords = ["gaming"]  # Replace with e.g. ["software development", "gaming"] as needed
desc_df = filter_by_keywords(df_selected, 'tool_description', desc_keywords)
desc_df.info()


# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('filtered_tools2.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 47 to 19181
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           45 non-null     int64 
 1   use_original      31 non-null     object
 2   use_aggregated    31 non-null     object
 3   name_tool         45 non-null     object
 4   tool_url_y        45 non-null     object
 5   tool_description  45 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


In [148]:
# 5. Add missing tool_ids to filtered_df (if needed)
tool_ids_to_add = [
    13653, 13360, 6052, 15514, 5467, 13189, 11563, 4853, 8901, 5013, 9134, 1843,
    10731, 19342, 6126, 1385, 469, 19680, 11257, 1474, 8419
]
filtered_df_gaming = add_missing_tool_ids(filtered_df_gaming, df_selected, tool_ids_to_add)


In [150]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           55 non-null     int64 
 1   use_original      53 non-null     object
 2   use_aggregated    53 non-null     object
 3   name_tool         55 non-null     object
 4   tool_url_y        55 non-null     object
 5   tool_description  55 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB


In [138]:
filtered_df.to_excel('gaming_sub_ranking.xlsx', index=False)

software development,


energy monitoring & analysis, energy management, climate control

gaming

fashion recommendation

food & drinks recommendations

receipt management

drone management

disease detection, medical image analysis

fall prevention

recruiting & hiring

legal assistance

collision detection & avoidance ( https://www.spoor.ai/collision-detection)

damage detection

car damage detection

travel planning

# ai ethics and governance



In [76]:

# Define your keywords and tool_ids for any use case
keywords = [
    "fall prevention"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids
#keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df = filter_by_keywords(df_selected, 'use_original', keywords)

## Medical image & disease detection

In [102]:
# Define your keywords and tool_ids for any use case
keywords = [
    "medical image analysis", "disease detection"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
tool_ids_to_edit = [6772, 3068, 10408, 13407, 8413, 7127, 1384, 1395, 24355, 6363]  # Example tool_ids
keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df = filter_by_keywords(df_selected, 'use_original', keywords)

In [105]:
# 2. Remove keywords from 'use_aggregated' for certain tool_ids
filtered_df = remove_keywords_from_column(filtered_df, 'use_aggregated', keywords_to_remove, tool_ids=tool_ids_to_edit)

In [110]:
# Replace 'disease detection' with 'medical disease detection' in 'use_aggregated' column of filtered_df
filtered_df['use_aggregated'] = filtered_df['use_aggregated'].str.replace(
    'disease detection', 'medical disease detection', regex=False
)


/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_59086/2578463729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['use_aggregated'] = filtered_df['use_aggregated'].str.replace(


In [119]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["disease"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(filtered_df['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('filtered_tools3_disease.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 354 to 19050
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           24 non-null     int64 
 1   use_original      17 non-null     object
 2   use_aggregated    17 non-null     object
 3   name_tool         24 non-null     object
 4   tool_url_x        24 non-null     object
 5   tool_description  24 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.3+ KB


In [115]:
desc_df.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
383,6775,;digital & ai-driven therapy;therapy managemen...,digital & ai-driven therapy;therapy management...,Insitro,https://insitro.com,insitro employs machine learning to integrate ...
408,6639,;disease detection;clinical trial management,;medical disease detection;clinical trial mana...,InSilico Medicine,https://insilico.com,PHARMA.AI integrates cutting-edge generative A...
412,6726,;disease detection;disease treatment;patient c...,;medical disease detection;disease treatment;p...,Viz.ai,https://viz.ai,Viz.ai offers an FDA-cleared AI-powered care c...
3687,6772,;climate & weather analysis;disease detection;...,climate & weather analysis;farm management & o...,Wolkus HQ,https://fasal.co,Fasal leverages IoT and AI to offer data-drive...
4777,3068,;disease detection;climate & weather analysis;...,climate & weather analysis;satellite image ana...,Agrio,https://agrio.app,Agrio is an app that uses AI algorithms to pro...


In [108]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 383 to 19104
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           42 non-null     int64 
 1   use_original      42 non-null     object
 2   use_aggregated    42 non-null     object
 3   name_tool         42 non-null     object
 4   tool_url_x        42 non-null     object
 5   tool_description  41 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.3+ KB


In [111]:
filtered_df.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
383,6775,;digital & ai-driven therapy;therapy managemen...,digital & ai-driven therapy;therapy management...,Insitro,https://insitro.com,insitro employs machine learning to integrate ...
408,6639,;disease detection;clinical trial management,;medical disease detection;clinical trial mana...,InSilico Medicine,https://insilico.com,PHARMA.AI integrates cutting-edge generative A...
412,6726,;disease detection;disease treatment;patient c...,;medical disease detection;disease treatment;p...,Viz.ai,https://viz.ai,Viz.ai offers an FDA-cleared AI-powered care c...
1109,13571,;disease detection;deep learning,;medical disease detection;deep learning,PMcardio,https://powerfulmedical.com,PMcardio's AI-powered platform enables rapid c...
1329,12910,;speech recognition;natural language processin...,;speech recognition;natural language processin...,Defined.ai,https://defined.ai,Defined.ai delivers ethically sourced and high...


In [112]:
# Save the filtered DataFrame to an Excel file
filtered_df.to_excel("disease.xlsx", index=False)

In [91]:

# Define your keywords and tool_ids for any use case
keywords = [
    "medical image analysis", "disease detection"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids
#keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df = filter_by_keywords(df_selected, 'use_original', keywords)

## Background Removal 

In [ ]:

# Define your keywords and tool_ids for any use case
keywords = [
    "background removal"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids

# 1. Filter by keywords in 'use_original'
bg_removal = filter_by_keywords(df_selected, 'use_original', keywords)

In [16]:
bg_removal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           176 non-null    int64 
 1   use_original      173 non-null    object
 2   use_aggregated    170 non-null    object
 3   name_tool         176 non-null    object
 4   tool_url_y        176 non-null    object
 5   tool_description  176 non-null    object
dtypes: int64(1), object(5)
memory usage: 8.4+ KB


In [9]:
bg_removal.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
54,13666,;image generation;background removal;logo desi...,;image generation;background removal;logo desi...,Kittl,kittl.com,Kittl is a browser-based design tool offering ...
143,18802,;image resizing;image color editing;background...,;image editing;image color editing;background ...,PicLumen,piclumen.com,PicLumen is an AI-driven image creation platfo...
152,5684,;image generation;background removal;image edi...,;image generation;background removal;image edi...,Recraft,recraft.ai,Recraft offers a premium image generation and ...
203,13467,;background removal,background removal,Remove.bg,remove.bg,remove.bg automatically removes image backgrou...
241,12663,;icon generation;image resizing;background rem...,;icon generation;image editing;background remo...,Icons8,icons8.com,"Icons8 provides apps and tools for design, off..."


In [13]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["background removal"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(bg_removal['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('background_removal.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 250 to 4418
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           3 non-null      int64 
 1   use_original      0 non-null      object
 2   use_aggregated    0 non-null      object
 3   name_tool         3 non-null      object
 4   tool_url_y        3 non-null      object
 5   tool_description  3 non-null      object
dtypes: int64(1), object(5)
memory usage: 168.0+ bytes


In [ ]:
tool_ids_to_add = [21308, 5342, 18266]  # Example tool_ids
# 5. Add missing tool_ids to filtered_df (if needed)

bg_removal = add_missing_tool_ids(bg_removal, df_selected, tool_ids_to_add)

In [43]:
import os, json, openai
from pydantic import BaseModel, ValidationError

openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

# 1️⃣  Pydantic schema
class BGCheck(BaseModel):
    background_removal: bool
    reason: str | None = None

# 2️⃣  Structured‑output request
SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = "Does this tool do *background removal*?\n\n\"\"\"{desc}\"\"\""

def check_bg_removal(desc: str) -> BGCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "bg_check",
            "description": "Return whether tool performs background removal",
            "parameters": BGCheck.model_json_schema()   #  ← auto‑build JSON Schema
        }],
        function_call={"name": "bg_check"}              #  ← ask for that output
    )

    args_json = completion.choices[0].message.function_call.arguments
    try:
        return BGCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

# --- quick demo ---------------------------
if __name__ == "__main__":
    sample = "Online editor that instantly removes image backgrounds."
    result = check_bg_removal(sample)
    print(result.model_dump())        # {'background_removal': True, 'reason': None}


{'background_removal': True, 'reason': 'Online editor that instantly removes image backgrounds.'}


In [18]:
# ⬇️ assumes you already executed the earlier cell that defines `check_bg_removal`
import pandas as pd

# df is the DataFrame you showed:
# Columns: tool_id, use_original, use_aggregated, name_tool, tool_url_y, tool_description

# 1. Call the LLM once per row and get a dict back
results = (
    bg_removal["tool_description"]
    .apply(lambda desc: check_bg_removal(desc).model_dump())  # → {"background_removal":…, "reason": …}
)

# 2. Expand that dict into two columns and attach them in one line
bg_removal[["background_removal", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(bg_removal.head())


   tool_id                                       use_original  \
0    13666  ;image generation;background removal;logo desi...   
1    18802  ;image resizing;image color editing;background...   
2     5684  ;image generation;background removal;image edi...   
3    13467                                ;background removal   
4    12663  ;icon generation;image resizing;background rem...   

                                      use_aggregated  name_tool    tool_url_y  \
0  ;image generation;background removal;logo desi...      Kittl     kittl.com   
1  ;image editing;image color editing;background ...   PicLumen  piclumen.com   
2  ;image generation;background removal;image edi...    Recraft    recraft.ai   
3                                 background removal  Remove.bg     remove.bg   
4  ;icon generation;image editing;background remo...     Icons8    icons8.com   

                                    tool_description  background_removal  \
0  Kittl is a browser-based design tool offeri

In [20]:
TARGET_USE = "background removal"

def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])

    if row["background_removal"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]

    # Keep the original style: leading ';' if the cell started with one
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

bg_removal["use_aggregated"] = bg_removal.apply(update_use_aggregated, axis=1)


In [46]:
SEARCH_SYSTEM = (
    "You are an analyst with access to a web_search tool. "
    "Search only enough to decide whether the given tool really performs "
    "background removal. Return a JSON dict with keys:\n"
    "  supports  – bool\n"
    "  evidence  – one‑sentence citation‑ready summary"
)

def verify_with_web(tool_name: str, tool_url: str) -> tuple[bool, str]:
    """Ask the search‑preview model for confirmation/evidence."""
    completion = openai.chat.completions.create(
        model="gpt-4o-mini-search-preview",
        temperature=0,
        tools=[{"type": "web_search"}],   # built‑in tool
        #tool_choice="auto",
        messages=[
            {"role": "system", "content": SEARCH_SYSTEM},
            {"role": "user",
             "content": (
                 f"Tool name: {tool_name}\n"
                 f"Homepage: {tool_url}\n\n"
                 "Decide if it offers *image background removal*. "
                 "Use web_search if needed. "
                 "Respond with JSON {{\"supports\": <bool>, \"evidence\": \"...\"}}."
             )
            }
        ],
    )

    resp = completion.choices[0].message.content
    data = json.loads(resp)
    return bool(data["supports"]), str(data["evidence"])


In [47]:
def robust_bg_check(row) -> BGCheck:
    """Run the 1st‑pass LLM; if it says False, verify with search."""
    first = check_bg_removal(row["tool_description"])

    if first.background_removal:
        return first            # already positive ⇒ trust it

    # First pass said “False” – verify
    supports, evidence = verify_with_web(row["name_tool"], row["tool_url_y"])

    if supports:
        return BGCheck(
            background_removal=True,
            reason=evidence,     # web evidence
            verified=True
        )
    else:
        # keep original “False” but mark that we actually checked
        first.verified = True
        first.reason = (first.reason or "") + " (double‑checked on the web)"
        return first


In [48]:
# Call once per row, just like before
results = (
    bg_removal
    .apply(robust_bg_check, axis=1)
    .apply(lambda r: r.model_dump())        # → dict
)

bg_removal[["background_removal", "reason", "verified"]] = results.apply(pd.Series)


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].function'.", 'type': 'invalid_request_error', 'param': 'tools[0].function', 'code': 'missing_required_parameter'}}

In [22]:
bg_removal.to_excel("bg_removal.xlsx", index=False)

In [21]:
bg_removal.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description,background_removal,reason
0,13666,;image generation;background removal;logo desi...,;image generation;logo design;text writing;ima...,Kittl,kittl.com,Kittl is a browser-based design tool offering ...,False,The description does not mention background re...
1,18802,;image resizing;image color editing;background...,;image editing;image color editing;character d...,PicLumen,piclumen.com,PicLumen is an AI-driven image creation platfo...,False,The description does not mention background re...
2,5684,;image generation;background removal;image edi...,;image generation;image editing;mockup generation,Recraft,recraft.ai,Recraft offers a premium image generation and ...,False,The description does not mention background re...
3,13467,;background removal,background removal,Remove.bg,remove.bg,remove.bg automatically removes image backgrou...,True,The description indicates that remove.bg autom...
4,12663,;icon generation;image resizing;background rem...,;icon generation;image editing;face swap;profi...,Icons8,icons8.com,"Icons8 provides apps and tools for design, off...",False,The description does not mention background re...


In [ ]:
## Background Removal 

## Validating autonomous vehicles & transport

In [34]:

# Define your keywords and tool_ids for any use case
keywords = [
    "autonomous vehicles & transport"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids

# 1. Filter by keywords in 'use_original'
auto_vehicles = filter_by_keywords(df_selected, 'use_aggregated', keywords)

In [35]:
auto_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 125 to 18558
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           20 non-null     int64 
 1   use_original      20 non-null     object
 2   use_aggregated    20 non-null     object
 3   name_tool         20 non-null     object
 4   tool_url_y        20 non-null     object
 5   tool_description  20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB


In [ ]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["background removal"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(bg_removal['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('background_removal.xlsx', index=False)


In [50]:
import os, json, openai
from pydantic import BaseModel, ValidationError

openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

# 1️⃣  Pydantic schema
class BGCheck(BaseModel):
    autonomous_vehicle_transport: bool
    reason: str | None = None

# 2️⃣  Structured‑output request
SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = "Does this tool or company do *autonomous vehicle & transport*, or is very closely related to this use case or business?\n\n\"\"\"{desc}\"\"\""

def check_desc(desc: str) -> BGCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "autonomous",
            "description": "Return whether tool performs autonomous vehicle & transport",
            "parameters": BGCheck.model_json_schema()   #  ← auto‑build JSON Schema
        }],
        function_call={"name": "autonomous"}              #  ← ask for that output
    )

    args_json = completion.choices[0].message.function_call.arguments
    try:
        return BGCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

# --- quick demo ---------------------------
if __name__ == "__main__":
    sample = "Shield AI builds autonomous systems for drones and aircraft, focusing on the Hivemind AI pilot to enable operations without GPS or human pilots, providing innovative solutions for both military and commercial use."
    result = check_desc(sample)
    print(result.model_dump())        # {'background_removal': True, 'reason': None}


{'autonomous_vehicle_transport': True, 'reason': 'Shield AI develops autonomous systems for drones and aircraft, which are closely related to autonomous vehicle and transport technologies.'}


In [51]:
# ⬇️ assumes you already executed the earlier cell that defines `check_bg_removal`
import pandas as pd

# df is the DataFrame you showed:
# Columns: tool_id, use_original, use_aggregated, name_tool, tool_url_y, tool_description

# 1. Call the LLM once per row and get a dict back
results = (
    auto_vehicles["tool_description"]
    .apply(lambda desc: check_desc(desc).model_dump())  # → {"background_removal":…, "reason": …}
)

# 2. Expand that dict into two columns and attach them in one line
auto_vehicles[["autonomous_vehicle_transport", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(auto_vehicles.head())


      tool_id                                     use_original  \
125     12158                 ;autonomous vehicles & transport   
126     12140                 ;autonomous vehicles & transport   
136     12153                 ;autonomous vehicles & transport   
537      6721                                  ;model training   
1032    13067  ;predictive analytics;sensor fusion;ai copilots   

                                         use_aggregated  name_tool tool_url_y  \
125                     autonomous vehicles & transport    Pony.ai    pony.ai   
126                     autonomous vehicles & transport  Shield AI  shield.ai   
136                     autonomous vehicles & transport       Nuro    nuro.ai   
537      model training;autonomous vehicles & transport     PlusAI    plus.ai   
1032  predictive analytics;sensor fusion;ai copilots...    Helm.ai    helm.ai   

                                       tool_description  \
125   Pony.ai is a global leader in autonomous drivi...  

/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_50777/1554959920.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auto_vehicles[["autonomous_vehicle_transport", "reason"]] = results.apply(pd.Series)


In [52]:
auto_vehicles.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description,autonomous_vehicle_transport,reason
125,12158,;autonomous vehicles & transport,autonomous vehicles & transport,Pony.ai,pony.ai,Pony.ai is a global leader in autonomous drivi...,True,Pony.ai specializes in autonomous driving tech...
126,12140,;autonomous vehicles & transport,autonomous vehicles & transport,Shield AI,shield.ai,Shield AI builds autonomous systems for drones...,True,Shield AI develops autonomous systems for dron...
136,12153,;autonomous vehicles & transport,autonomous vehicles & transport,Nuro,nuro.ai,Nuro offers AI-first autonomous driving techno...,True,Nuro specializes in AI-first autonomous drivin...
537,6721,;model training,model training;autonomous vehicles & transport,PlusAI,plus.ai,Plus revolutionizes autonomous driving with a ...,True,The company focuses on autonomous driving and ...
1032,13067,;predictive analytics;sensor fusion;ai copilots,predictive analytics;sensor fusion;ai copilots...,Helm.ai,helm.ai,Helm.ai provides AI solutions for ADAS to L4 a...,True,Helm.ai provides AI solutions specifically for...


In [ ]:
TARGET_USE = "background removal"

def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])

    if row["background_removal"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]

    # Keep the original style: leading ';' if the cell started with one
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

bg_removal["use_aggregated"] = bg_removal.apply(update_use_aggregated, axis=1)


In [ ]:
bg_removal.to_excel("bg_removal.xlsx", index=False)

In [ ]:
class BGCheck(BaseModel):
    background_removal: bool
    reason: str | None = None
    verified: bool = False          # ← True = second‑pass verification ran


## 